# 导入库

In [20]:
# -*- coding: utf-8 -*-
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing  # 预处理模块
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import time

import warnings
warnings.filterwarnings("ignore")

# 数据导入与预处理

#### 数据导入

In [3]:
#定义加载数据集的方法
def load_dataset(path,inputfile):
    return pd.read_csv(path+inputfile)

In [6]:
path='data/zombie_enterprise_classification/'
#加载训练集
df_base=load_dataset(path,'base-train.csv')
df_knowledge=load_dataset(path,'knowledge-train.csv')
df_money=load_dataset(path,'money-train.csv')
df_year=load_dataset(path,'year-train.csv')

#加载无标签数据集
df_baseT=load_dataset(path,'base-test.csv')
df_knowledgeT=load_dataset(path,'knowledge-test.csv')
df_moneyT=load_dataset(path,'money-test.csv')
df_yearT=load_dataset(path,'year-test.csv')

#### 数据维度

In [8]:
print('base_train:',df_base.shape)
print('know_train:',df_knowledge.shape)
print('money_train',df_money.shape)
print('year_train',df_year.shape)
print('base_test:',df_baseT.shape)
print('know_test:',df_knowledgeT.shape)
print('money_test',df_moneyT.shape)
print('year_test',df_yearT.shape)

base_train: (28519, 9)
know_train: (28519, 4)
money_train (85548, 10)
year_train (85548, 10)
base_test: (7132, 8)
know_test: (7132, 4)
money_test (21396, 10)
year_test (21396, 10)


#### 按年份拆分money和year数据集
- money数据集拆分为money2015,money2016,money2016三个不同的数据集  
- year数据集拆分为year2015,year2016,year2016三个不同的数据集

In [9]:
#定义拆分数据集的方法
def split_dataset_by(df,col='year'):
    """
    df：要处理的数据集的dataframe
    col：要按哪一列处理
    return:拆分后的数据集dataframe组成的列表
    """
    split_df=[]
    col_values=df[col].value_counts().index.values#col这一列的唯一值
    columns=df.columns.values.tolist()#df所有列名
    columns.remove(col)#删除掉要拆分的这一列的名称col
    #循环拆分数据集
    for col_value in col_values:
        col_value=int(col_value)
        locals()[col+str(col_value)]=df[df[col].isin([col_value])].drop([col],axis=1)
        new_columns=['ID']
        for column in columns:
            if column=='ID':
                continue
            new_columns.append(column+str(col_value))
        locals()[col+str(col_value)].columns=new_columns#拆分出来的数据集重新命名它的列名称
        split_df.append(locals()[col+str(col_value)])#处理完成的数据集添加到列表中
    return split_df

In [10]:
#按年份拆分money-train,year-train,money-test,year-test四个数据集
split_money_train=split_dataset_by(df_money)
split_year_train=split_dataset_by(df_year)
split_money_test=split_dataset_by(df_moneyT)
split_year_test=split_dataset_by(df_yearT)

#### 合并数据集

In [11]:
#定义合并数据集的方法
def merge_dataset(dfs,col='ID'):
    """
    df：要处理的数据集的dataframe
    col：要按哪一列处理
    return:合并后的数据集dataframe
    """
    df_res=pd.DataFrame()
    for df in dfs:
        if len(df_res)==0:
            df_res=df
        else:
            df_res=pd.merge(df_res,df,on=col,how='outer')#按col列合并，outer表示保留两个表的信息
    return df_res

In [12]:
#分别合并money_train,year_train拆分后的数据集
money_train_res=merge_dataset(split_money_train)
year_train_res=merge_dataset(split_year_train)
money_test_res=merge_dataset(split_money_test)
year_test_res=merge_dataset(split_year_test)
#合并数据集为train,test
train=merge_dataset([df_base,df_knowledge,money_train_res,year_train_res])
test=merge_dataset([df_baseT,df_knowledgeT,money_test_res,year_test_res])

In [13]:
#查看合并后的数据集
train.head()

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,flag,专利,...,纳税总额2015,所有者权益合计2015,从业人数2017,资产总额2017,负债总额2017,营业总收入2017,主营业务收入2017,利润总额2017,纳税总额2017,所有者权益合计2017
0,5986361,2014.0,7090.0,服务业,湖北,有限责任公司,自然人,0.93,0,0.0,...,0.0,14180.0,503.0,85080.0,155980.0,187176.0,149740.8,74870.4,37435.2,-70900.0
1,5991749,2007.0,5940.0,零售业,湖南,合伙企业,企业法人,0.57,0,1.0,...,270864.0,-225720.0,514.0,178200.0,344520.0,267300.0,106920.0,80190.0,160380.0,-166320.0
2,5998154,2002.0,9720.0,工业,福建,合伙企业,自然人,0.74,0,1.0,...,101088.0,9720.0,891.0,729000.0,719280.0,1458000.0,729000.0,729000.0,291600.0,NaN
3,5984390,2000.0,4800.0,商业服务业,山东,股份有限公司,NaN,0.90,0,0.0,...,639360.0,146400.0,819.0,326400.0,NaN,163200.0,65280.0,48960.0,65280.0,165600.0
4,5980535,2004.0,4530.0,零售业,广东,农民专业合作社,自然人,0.95,0,0.0,...,27904.8,52095.0,158.0,176670.0,258210.0,494676.0,296805.6,197870.4,0.0,-81540.0


#### 缺失值处理

In [14]:
#缺失值分布统计
train.isnull().sum()

ID                   0
注册时间               289
注册资本               299
行业                 254
区域                 278
企业类型               292
控制人类型              293
控制人持股比例            296
flag                 0
专利                 286
商标                 303
著作权                282
债权融资额度2017         523
债权融资成本2017         544
股权融资额度2017         562
股权融资成本2017         556
内部融资和贸易融资额度2017    528
内部融资和贸易融资成本2017    531
项目融资和政策融资额度2017    561
项目融资和政策融资成本2017    575
债权融资额度2016         550
债权融资成本2016         574
股权融资额度2016         554
股权融资成本2016         583
内部融资和贸易融资额度2016    567
内部融资和贸易融资成本2016    547
项目融资和政策融资额度2016    551
项目融资和政策融资成本2016    570
债权融资额度2015         585
债权融资成本2015         609
股权融资额度2015         577
股权融资成本2015         584
内部融资和贸易融资额度2015    561
内部融资和贸易融资成本2015    598
项目融资和政策融资额度2015    594
项目融资和政策融资成本2015    550
从业人数2016           542
资产总额2016           594
负债总额2016           561
营业总收入2016          547
主营业务收入2016         546
利润总额2016           541
纳税总额2016           556
所有者权益合计2016

In [15]:
#前向填充，默认参数按行来，即使用上一行的值
train=train.fillna(method='ffill')
test=test.fillna(method='ffill')

#### 标签编码
LabelEncoder是用来对分类型特征值进行编码，即对不连续的数值或文本进行编码  
- fit(y) ：fit可看做一本空字典，y可看作要塞到字典中的词。
- fit_transform(y)：相当于先进行fit再进行transform，即把y塞到字典中去以后再进行transform得到索引值。

In [18]:
#待编码的属性
cols_encode=['注册时间','行业','区域','企业类型','控制人类型']
#将数值类型（数值作为类型的列）转化为字符串类型
feature=['注册时间']
for i in feature:
    train[i]=train[i].astype(str)

In [22]:
#对不连续的数字或文本进行编号，转换成连续的数值型变量
for c in cols_encode:
    le=LabelEncoder()
    train[c]=le.fit_transform(list(train[c].values))
    test[c]=le.fit_transform(list(test[c].values))

In [24]:
#查看编码后的结果
test.head()

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,专利,商标,...,纳税总额2016,所有者权益合计2016,从业人数2015,资产总额2015,负债总额2015,营业总收入2015,主营业务收入2015,利润总额2015,纳税总额2015,所有者权益合计2015
0,5991927,10,8790.0,2,5,3,0,0.64,0.0,0.0,...,7383.6,30765.0,417.0,131850.0,61530.0,118665.0,59332.5,47466.0,47466.0,70320.0
1,5998351,5,270.0,3,0,3,1,0.59,1.0,0.0,...,6058.8,-5400.0,494.0,5400.0,5130.0,1620.0,648.0,324.0,324.0,270.0
2,5992703,12,230.0,3,4,0,0,0.52,1.0,1.0,...,2760.0,-2300.0,575.0,3450.0,6440.0,5865.0,2346.0,1759.5,1173.0,-2990.0
3,5979231,3,5980.0,1,1,1,0,0.92,0.0,0.0,...,223891.2,-143520.0,309.0,143520.0,275080.0,71760.0,50232.0,35880.0,43056.0,-131560.0
4,5995422,3,160.0,2,0,2,1,0.70,0.0,1.0,...,2419.2,160.0,485.0,4160.0,2000.0,5824.0,2329.6,2329.6,0.0,2160.0


#### 训练集将flag属性放到最后一列

In [25]:
#列名
current_columns=train.columns.drop('flag').values.tolist()
current_columns.append('flag')
#改变列顺序为cols
train=train.loc[:,current_columns] #获取所有的行，按照cols列表来依次取原df的列，得到新的DataFrame赋给新的变量df

In [29]:
#将预测数据集和训练集属性对齐
test=test.loc[:,current_columns[:-1]]

In [30]:
test.head()

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,专利,商标,...,纳税总额2015,所有者权益合计2015,从业人数2017,资产总额2017,负债总额2017,营业总收入2017,主营业务收入2017,利润总额2017,纳税总额2017,所有者权益合计2017
0,5991927,10,8790.0,2,5,3,0,0.64,0.0,0.0,...,47466.0,70320.0,778.0,307650.0,448290.0,1138305.0,682983.0,341491.5,0.0,-140640.0
1,5998351,5,270.0,3,0,3,1,0.59,1.0,0.0,...,324.0,270.0,799.0,6480.0,9315.0,31104.0,24883.2,3110.4,18662.4,-2835.0
2,5992703,12,230.0,3,4,0,0,0.52,1.0,1.0,...,1173.0,-2990.0,80.0,2300.0,4140.0,2070.0,1656.0,207.0,414.0,-1840.0
3,5979231,3,5980.0,1,1,1,0,0.92,0.0,0.0,...,43056.0,-131560.0,25.0,251160.0,367770.0,251160.0,200928.0,25116.0,414.0,-116610.0
4,5995422,3,160.0,2,0,2,1,0.70,0.0,1.0,...,0.0,2160.0,589.0,7200.0,7040.0,29520.0,17712.0,2952.0,5904.0,160.0


# 模型定义与训练

#### 获得训练向量$X(n\_samples, n\_features)$和目标值$y(n\_samples)$

- 标准化   
    1. 当输入数据集的特征在它们的范围之间具有大差异时，或者它们各自使用的单位不同时（比如说一些用米，一些用厘米），我们会想到对数据进行标准化。  
    2. 这些初始特征范围的差异，会给许多机器学习模型带来不必要的麻烦。例如，对于基于距离计算的模型来说，当其中一个特征值变化范围较大时，那么预测结果很大程度上就会受到它的影响。  
    3. 举一个例子。有一个二维的数据集，它有两个特征，以米为单位的高度（范围是 1 到 2 米）和以磅为单位的重量（范围是 10 到 200 磅）。无论在这个数据集上使用什么基于距离的模型，重量特征对结果的影响都会大大的高于高度特征，因为它的数据变化范围相对更大。因此，为了预防这种问题的发生，会用到数据标准化来约束重量特征的数据变化范围。 
- 方法
    1. 将数据按期属性（按列进行）减去其均值，并处以其方差。得到的结果是，对于每个属性/每列来说所有数据都聚集在0附近，方差为1。  
    2. 使用`sklearn.preprocessing.StandardScaler`类，使用该类的好处在于可以保存训练集中的参数（均值、方差）直接使用其对象转换测试集数据。

In [31]:
#训练集向量
values=train.values
X=values[:,1:-1]
y=values[:,-1]

#预测数据集向量
valuesT=test.values
X_t=valuesT[:,1:]

#按8:2分割数据集为训练集和测试集，固定随机种子
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2020)

#对数据集进行标准化
scaler=preprocessing.StandardScaler().fit(X_train)#通过训练集获得归一化函数

#使用相同的归一化函数处理训练向量和测试向量，以及无标签测试向量
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)
X_t=scaler.transform(X_t)

#训练集中正例和负例个数
y_list=y_train.tolist()
print("正例：",y_list.count(0))
print("负例：",y_list.count(1))

正例： 13933
负例： 8882


#### 训练  
使用SVM作为模型时，通常采用如下流程：  

- 对样本数据进行归一化
- 应用核函数对样本进行映射
- 用cross-validation和grid-search对超参数进行优选
- 用最优参数训练得到模型
- 测试

In [164]:
#SVM rbf核函数，C，gamma参数调优
grid = GridSearchCV(SVC(), param_grid={"C":[0.1, 1, 10], "gamma":[1, 0.1, 0.01]}, cv=4)  
grid.fit(X_train, y_train)  
print("The best parameters are %s with a score of %0.2f" %(grid.best_params_, grid.best_score_))

The best parameters are {'C': 10, 'gamma': 0.01} with a score of 0.96


In [32]:
#SVM模型训练
clf=SVC(C=10,gamma=0.01)
t1=time.time()
clf.fit(X_train,y_train)
t2=time.time()
print("训练时间： %fs"%(t2-t1))
sv=clf.n_support_
print("支持向量：",sv)

#支持向量中正例负例个数
print("正例：",sv[0])
print("负例：",sv[1])

训练时间： 15.639801s
支持向量： [1961 2786]
正例： 1961
负例： 2786


In [33]:
accur=clf.score(X_test,y_test)
print("准确率:",accur)

准确率: 0.9680925666199158


# 预测无标签数据集

#### 预测标签向量

In [34]:
#预测标签向量
y_pre=clf.predict(X_t).tolist()

#### 添加id列，输出结果

In [41]:
df_res=pd.DataFrame()
ids=test['ID'].values.tolist()
df_res['ID']=ids
df_res['flag']=y_pre
df_res=df_res.astype(int)
df_res.to_csv("output/res.txt",index=False)#输出到文件，不加行索引 
df_res

,ID,flag
0,5991927,0
1,5998351,0
2,5992703,1
3,5979231,0
4,5995422,1
...,...,...
7127,80564,1
7128,978515,1
7129,568065,1
7130,1889883,1
